# quasi-Treffz DG for elliptic PDEs
$$
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\bK}{\boldsymbol{K}}
\newcommand{\bx}{\boldsymbol{x}}
\newcommand{\N}[1]{\left\|#1\right\|}
$$
Consider the following diffusion-advection-reaction BVP
$$
\begin{cases}
\mathcal{L}u:=\text{div}(-\bK \nabla u +\bbeta u) +\sigma u = f   &\text{in $\Omega$,}\\
u=g_{D} &\text{on $ \Gamma_{D}$,}\\
- \bK \nabla u  \cdot \mathbf{n} =g_{N}  &\text{on $ \Gamma_{N}$.}
\end{cases}
$$
with $ \bK=\bK^T$ diffusion matrix, $\bbeta$ advection vector and $\sigma$ reaction scalar.

We want to solve this second-order elliptic boundary value problem using a quasi-Trefftz DG method.

In [ ]:
from ngsolve import *
from ngstrefftz import *
from netgen.occ import *

### Constructing a quasi-Trefftz space

$$\newcommand{\IN}{\mathbb{N}}\newcommand{\IR}{\mathbb{R}}
\newcommand{\IP}{\mathbb{P}}\newcommand{\IT}{\mathbb{T}}
\newcommand{\QT}{{\mathbb{Q\!T}}}
\newcommand{\calM}{{\mathcal M}}
\newcommand{\bi}{{\boldsymbol i}}
\newcommand{\bx}{{\boldsymbol x}}$$
A polynomial quasi-Trefftz space for the diffusion-advection-reaction equation is given by the polynomials that locally belongs to
$$
\QT^p_f(E):=\big\{ v\in \IP^p(E) \mid D^{\bi} \mathcal{L} v (\bx^E)=D^{\bi} f (\bx^E)\quad \forall \bi\in \IN^d_0,\ |\bi|\leq p-m\big\}
\qquad p\in \mathbb{N}.
$$

We can construct it in NGSolve like so:

In [ ]:
h = 0.03; 
order = 3;
mesh = Mesh(unit_square.GenerateMesh(maxh=h))
with TaskManager():
    fes = trefftzfespace(mesh,order=order,eq="qtelliptic")

Using the eq key word one needs to tell `trefftzfespace` the operator for which to construct quasi-Trefftz functions.

We will consider the following manufactured example

In [ ]:
exact = sin(pi*(x+y))
K = CF((1+x+y,0,0,1+x+y),dims=(2,2))
beta = CF((1,0))
sigma = 3/(1+x+y)
Dbndc = exact
rhs = -sum( (K*CF((exact.Diff(x),exact.Diff(y))))[i].Diff(var) for i,var in enumerate([x,y])) + beta*CF((exact.Diff(x),exact.Diff(y))) + sigma*exact

To set the coefficients for the construction of the quasi-Trefftz basis functions use 

In [ ]:
with TaskManager():
    fes.SetCoeff(K,beta,sigma)

To set the right-hand side for the construction of the quasi-Trefftz particular approximate solution use 

In [ ]:
with TaskManager():
    uf = fes.GetEWSolution(rhs)

$$
\newcommand{\calT}{\mathcal{T}}
\newcommand{\calF}{\mathcal{F}}
\newcommand{\calA}{\mathcal{A}}
\newcommand{\jmp}[1]{[\![#1]\!]}    
\newcommand{\mvl}[1]{\{\!\!\{#1\}\!\!\}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\bK}{\boldsymbol{K}}
\newcommand{\bx}{\boldsymbol{x}}
\newcommand{\bn}{\mathbf{x}}
\newcommand{\N}[1]{\left\|#1\right\|}
$$ 
A possbile DG method is
$$
\text{Find } u_h \in V_h \text{ such that }
\calA_h^\mathrm{dar}(u_h,v_h)=L_h(v_h) \quad \forall v_h \in V_h,
$$
with the DG bilinear form 
$$\begin{align}
\calA_h^\mathrm{dar}(u_h,v_h):=&\sum_{E\in \calT_h}	\int_E \Big(\Big(\bK\nabla u_h -\bbeta u_h\Big) \cdot \nabla  v_h+\sigma u_h  v_h\Big)\\&+
\sum_{F\in\calF_h^{\mathrm I}\cup \calF_h^{\mathrm D}}
\int_F \Big(-\mvl{\bK   \nabla u_h } \cdot \jmp{v_h}- \jmp{u_h} \cdot \mvl{\bK   \nabla v_h}  
+ \gamma\frac{K_F}{h_F} \jmp{u_h}\cdot \jmp{v_h}\Big)\\ 
&+
\int_{\mathcal{F}_h^{\mathrm I}} \Big( \mvl{\bbeta  w} \cdot \jmp{v_h}+\frac12|\bbeta\cdot \bn_F|\jmp{w}\cdot\jmp{v_h}\Big)+
\int_{\mathcal{F}_h^+} ( \bbeta  w) \cdot \bn v_h,
\end{align} $$
and the linear form
$$
L_h(v_h):=\sum_{E\in\calT_h}\int_E  f v_h - 
\int_{\mathcal{F}_h^\mathrm{N}}  g_{\mathrm N} v_h +
\int_{\mathcal{F}_h^{\mathrm D}} g_{\mathrm D}  \Big(- \bK   \nabla v_h  \cdot \bn  + \gamma\frac{K_F}{h_F} v_h%-   \bbeta  \cdot \bn v_h
\Big)
-\int_{\mathcal{F}_h^-}g_D\bbeta\cdot\bn v_h.
$$

In [ ]:
def dgell(fes,K,beta=CF((0,0)),sigma=0,Dbndc=0,Dbnd=".*",Nbndc=0,Nbnd="",rhs=0,uf=None,alpha=0):
    mesh = fes.mesh
    order = fes.globalorder

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    alpha = 50*order**2/h if alpha==0 else alpha

    u = fes.TrialFunction()
    v = fes.TestFunction()
    jump = lambda u: (u-u.Other())*n
    mean_d = lambda u: 0.5 * K * (grad(u)+grad(u).Other())
    mean_B = lambda u: 0.5 * beta * (u+u.Other())

    a = BilinearForm(fes)
    a += K*grad(u)*grad(v) * dx \
        +alpha*jump(u)*jump(v) * dx(skeleton=True) \
        +(-mean_d(u)*jump(v)-mean_d(v)*jump(u)) * dx(skeleton=True) \
        +alpha*u*v * ds(skeleton=True,definedon=mesh.Boundaries(Dbnd)) \
        +(-K*grad(u)*n*v-K*grad(v)*n*u)* ds(skeleton=True,definedon=mesh.Boundaries(Dbnd))
    a += (-beta*u*grad(v) + sigma*u*v) * dx \
        + (mean_B(u) * jump(v) + 0.5*sqrt((beta*n)**2)*jump(u)*jump(v)) * dx(skeleton=True) \
        + beta*u*n*v * ds(skeleton=True,definedon=mesh.Boundaries(Nbnd))

    f = LinearForm(fes)
    f += Dbndc * (-K*grad(v)*n + alpha*v - beta*n*v) * ds(skeleton=True,definedon=mesh.Boundaries(Dbnd)) \
         - Nbndc * v * ds(skeleton=True,definedon=mesh.Boundaries(Nbnd)) \
         + rhs*v*dx
    if uf:
        f += -K*grad(uf)*grad(v) * dx \
            -alpha*jump(uf)*jump(v) * dx(skeleton=True) \
            -(-mean_d(uf)*jump(v)-mean_d(v)*jump(uf)) * dx(skeleton=True) \
            -alpha*uf*v * ds(skeleton=True,definedon=mesh.Boundaries(Dbnd)) \
            -(-K*grad(uf)*n*v-K*grad(v)*n*uf)* ds(skeleton=True,definedon=mesh.Boundaries(Dbnd))
        f += (beta*uf*grad(v) - sigma*uf*v) * dx \
            - (mean_B(uf) * jump(v) + 0.5*sqrt((beta*n)**2)*jump(uf)*jump(v)) * dx(skeleton=True) \
            - beta*uf*n*v * ds(skeleton=True,definedon=mesh.Boundaries(Nbnd))

    with TaskManager():
        a.Assemble()
        f.Assemble()

    return a,f

Consider the previous manufactured example and obtain the approximate solution `gfu` 

In [ ]:

a,f = dgell(fes,K,beta,sigma,Dbndc=Dbndc,rhs=rhs,uf=uf)
gfu = GridFunction(fes)

with TaskManager():
    gfu.vec.data = a.mat.Inverse() * f.vec

We can calculate the $L^2(\Omega)$ error:

In [ ]:
error = sqrt(Integrate((gfu-exact)**2, mesh))
print("quasi-Trefftz error:",error)

We can plot the approximate solution like so:

In [ ]:
from ngsolve.webgui import Draw
Draw(gfu)

### Advection-dominated example

In [ ]:
j=3
nu = 10**(-j)
K = CF((nu,0,0,nu),dims=(2,2))
beta = CF((y+1,-x+2))
sigma = 0
Dbndc = IfPos(IfPos(x-10**-10,0,1) + IfPos(y-10**-10,0,1)*IfPos(x-1/3,0,1) - 0.5, 1, 0)
Nbndc = 0
Dbnd = "bottom|left"
Nbnd = "top|right"

In [ ]:
with TaskManager():
    fes.SetCoeff(K,beta,sigma)
    
alpha = nu*100/h
a,f = dgell(fes,K,beta,sigma,Dbndc=Dbndc,Dbnd=Dbnd,Nbnd=Nbnd,Nbndc=Nbndc,alpha=alpha)
  
gfu = GridFunction(fes)
with TaskManager():
    gfu.vec.data = a.mat.Inverse() * f.vec

In [ ]:
from ngsolve.webgui import Draw
Draw(gfu)

To see the 3D surface click `Open Controls` and select `deformation`.

### Circle obstacle example

In [ ]:
h = 0.05
order = 4
from netgen.occ import *
from ngsolve.webgui import Draw 
wp = WorkPlane().RectangleC(2,2) \
    .Circle(0,0,0.25).Reverse() 
geometry = wp.Face()
mesh = Mesh(OCCGeometry(geometry,dim=2).GenerateMesh(maxh=h))
mesh.Curve(order)
mesh.ngmesh.SetBCName(0,"top")
mesh.ngmesh.SetBCName(1,"bottom")
mesh.ngmesh.SetBCName(2,"left")
mesh.ngmesh.SetBCName(3,"right")
mesh.ngmesh.SetBCName(4,"innercircle")

In [ ]:
j = 3
nu = 10**(-j)
K = CF((nu,0,0,nu),dims=(2,2))
beta = CF((1+1/16*(y**2-x**2)/((x**2+y**2)**2),-1/8*x*y/((x**2+y**2)**2)))
sigma = 0
Dbndc = IfPos(x+1/2,1,0)
Nbndc = 0
Dbnd="right|innercircle"
Nbnd="left|top|bottom"
  
with TaskManager():
    fes = trefftzfespace(mesh,order=order,eq="qtelliptic")
    fes.SetCoeff(K,beta,sigma)
    
alpha = nu*50*order/h
a,f = dgell(fes,K,beta,sigma,Dbndc=Dbndc,Dbnd=Dbnd,Nbnd=Nbnd,Nbndc=Nbndc,alpha=alpha)  
gfu = GridFunction(fes)
with TaskManager():
    gfu.vec.data = a.mat.Inverse() * f.vec

Draw(gfu,mesh,"gfu")

### L-shaped domain example

In [ ]:
import netgen.geom2d as geom2d;
geo = geom2d.SplineGeometry()
p1 = geo.AppendPoint (1,1)
p2 = geo.AppendPoint (0,1)
p3 = geo.AppendPoint (0,0.5)
p4 = geo.AppendPoint (0.5,0.5)
p5 = geo.AppendPoint (0.5,0)
p6 = geo.AppendPoint (1,0)

geo.Append (["line", p1, p2], bc=1)
geo.Append (["line", p2, p3], bc=1)
geo.Append (["line", p3, p4], bc=1)
geo.Append (["line", p4, p5], bc=1)
geo.Append (["line", p5, p6], bc=1)
geo.Append (["line", p6, p1], bc=1)

h = 0.05;
mesh = Mesh( geo.GenerateMesh(maxh=h))

In [ ]:
j = 3
nu = 5*10**(-j)
K = CF((nu,0,0,nu),dims=(2,2))
beta = CF((-y,x))
sigma = 0
Dbndc = IfPos(y,0,1)

order = 4  
with TaskManager():
    fes = trefftzfespace(mesh,order=order,eq="qtelliptic")
    fes.SetCoeff(K,beta,sigma)
alpha = nu*50*order/h
a,f = dgell(fes,K,beta,sigma,Dbndc=Dbndc,alpha=alpha)
gfu = GridFunction(fes)
with TaskManager():
    gfu.vec.data = a.mat.Inverse() * f.vec

Draw(gfu,mesh,"gfu")